In [ ]:
from google.colab import drive
drive.mount('/content/drive')

# Set the paths for your training, validation, and test datasets
train_dir = '/content/drive/MyDrive/data/sdss_dataset_split/Train'
val_dir = '/content/drive/MyDrive/data/sdss_dataset_split/Validation'
test_dir = '/content/drive/MyDrive/data/sdss_dataset_split/Test'


Mounted at /content/drive


In [ ]:
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, Flatten, Dropout
from tensorflow.keras.applications import ResNet101
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import Adam, SGD
from tensorflow.keras.callbacks import ReduceLROnPlateau, ModelCheckpoint, EarlyStopping

# Define image size and batch size
img_height, img_width = 224, 224
batch_size = 32

# Load the ResNet101 model without the top layer
base_model = ResNet101(weights='imagenet', include_top=False, input_shape=(img_height, img_width, 3))

# Unfreeze the last few layers for fine-tuning
for layer in base_model.layers[-8:]:
    layer.trainable = True

# Add custom layers on top of the base model
x = Flatten()(base_model.output)
x = Dense(512, activation='relu')(x)
x = Dropout(0.5)(x)
x = Dense(3, activation='softmax')(x)

# Create the final model
model = Model(inputs=base_model.input, outputs=x)


171446536/171446536 ━━━━━━━━━━━━━━━━━━━━ 1s 0us/step


In [ ]:
# Compile the model with a learning rate and optimizer
optimizer = Adam(learning_rate=1e-5)  # You can switch to SGD if needed
model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])


In [ ]:
# Data augmentation and data generators for loading images
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=40,
    width_shift_range=0.3,
    height_shift_range=0.3,
    shear_range=0.3,
    zoom_range=0.3,
    brightness_range=[0.8, 1.2],
    horizontal_flip=True,
    vertical_flip=True,
    fill_mode='nearest'
)

val_datagen = ImageDataGenerator(rescale=1./255)
test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='categorical'
)

val_generator = val_datagen.flow_from_directory(
    val_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='categorical'
)

test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='categorical'
)


Found 5240 images belonging to 3 classes.
Found 650 images belonging to 3 classes.
Found 650 images belonging to 3 classes.


In [ ]:
# Callbacks for learning rate reduction, early stopping, and model checkpoint
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=3, min_lr=1e-7, verbose=1)
early_stopping = EarlyStopping(monitor='val_loss', patience=8, verbose=1, restore_best_weights=True)
checkpoint_filepath = '/content/drive/MyDrive/best_resnet101_model.keras'
model_checkpoint = ModelCheckpoint(
    filepath=checkpoint_filepath,
    monitor='val_accuracy',
    save_best_only=True,
    verbose=1
)

# Fit the model
history = model.fit(
    train_generator,
    epochs=50,  # Adjust epochs as needed
    validation_data=val_generator,
    callbacks=[reduce_lr, early_stopping, model_checkpoint]
)


Epoch 1/50
164/164 ━━━━━━━━━━━━━━━━━━━━ 0s 491ms/step - accuracy: 0.8335 - loss: 0.4218
Epoch 1: val_accuracy improved from -inf to 0.82462, saving model to /content/drive/MyDrive/best_resnet101_model.keras
164/164 ━━━━━━━━━━━━━━━━━━━━ 98s 574ms/step - accuracy: 0.8334 - loss: 0.4218 - val_accuracy: 0.8246 - val_loss: 0.4207 - learning_rate: 2.5000e-06
Epoch 2/50
164/164 ━━━━━━━━━━━━━━━━━━━━ 0s 500ms/step - accuracy: 0.8383 - loss: 0.4184
Epoch 2: val_accuracy improved from 0.82462 to 0.82615, saving model to /content/drive/MyDrive/best_resnet101_model.keras
164/164 ━━━━━━━━━━━━━━━━━━━━ 100s 580ms/step - accuracy: 0.8383 - loss: 0.4184 - val_accuracy: 0.8262 - val_loss: 0.4118 - learning_rate: 2.5000e-06
Epoch 3/50
164/164 ━━━━━━━━━━━━━━━━━━━━ 0s 490ms/step - accuracy: 0.8579 - loss: 0.3930
Epoch 3: val_accuracy improved from 0.82615 to 0.83077, saving model to /content/drive/MyDrive/best_resnet101_model.keras
164/164 ━━━━━━━━━━━━━━━━━━━━ 99s 574ms/step - accuracy: 0.8578 - loss: 0.393

In [ ]:
# Evaluate the model on the test data
test_loss, test_acc = model.evaluate(test_generator)
print(f'Test accuracy: {test_acc}')


In [ ]:
# Save the final model
final_model_filepath = '/content/drive/MyDrive/final_resnet101_model.keras'
model.save(final_model_filepath)


In [ ]:
# Load the saved model
model = tf.keras.models.load_model(final_model_filepath)

from tensorflow.keras.preprocessing import image
import numpy as np

# Load and preprocess an image
img_path = '/content/drive/MyDrive/data/sdss_dataset_split/Test/6/image_102_class_6.jpg'
img = image.load_img(img_path, target_size=(img_height, img_width))
img_array = image.img_to_array(img)
img_array = np.expand_dims(img_array, axis=0)
img_array = img_array / 255.0  # Normalize pixel values

# Make predictions
predictions = model.predict(img_array)

# Get the predicted class
predicted_class = np.argmax(predictions, axis=1)
print(f'Predicted class index: {predicted_class[0]}')

# Define class labels and map to predicted class
class_labels = ['Class 5', 'Class 6', 'Class 7']
predicted_label = class_labels[predicted_class[0]]
print(f'Predicted class label: {predicted_label}')
